<a href="https://colab.research.google.com/github/MLo7Ghinsan/DiffSinger_colab_notebook_MLo7/blob/main/SOFA_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # Mount Drive and install Conda
#@markdown Basically installs all necesarry things for the notebook to work <br/> **Wait until it restarts your session !**
#@markdown Mount Drive
from IPython.display import clear_output
from google.colab import drive

%cd /content
drive.mount('/content/drive')
!pip install -q condacolab
clear_output() # rawr

import condacolab
condacolab.install()

In [ ]:
#@title # Main installation

%cd /content
!conda create -n aligner python=3.8 -y
!git clone https://github.com/qiuqiao/SOFA.git
!git clone https://github.com/UtaUtaUtau/nnsvs-db-converter #for lab conversion
#converter stuff
!pip install numpy scipy soundfile librosa praat-parselmouth pydub
#SOFA stuff
!source activate aligner; \
pip install -r /content/SOFA/requirements.txt
!source activate aligner; \
pip install torch torchaudio praat-parselmouth onnx onnxsim scipy
!source activate aligner; \
pip install ipykernel


In [ ]:
#@title #Extract Data
#@markdown ___
%cd /content
#@markdown this cell will create a folder name [raw_data] in the root folder and extract your data into it

data_type = "full label (.lab)" # @param ["full label (.lab)", "weak label (.txt)", ".wav only"]

#@markdown <font size="-1.5"> The path to your data zip file

data_zip_path = "" #@param {type:"string"}

#@markdown <font size="-1.5"> Select this if you want to clear your raw data folder

clear_data = True # @param {type:"boolean"}

estimate_midi = False

segment_length = 30
max_silence_phoneme_amount = 60

# leaving -S at 60 so max silence can be 60 seconds that exceeds the segment legnth cap idk why///
# making the segment length cap at 35 secs because any longer than that would make training goes really slow

# my ass dont remember why i made two... i think one is unnecessary extra but mehhh

import os
import csv
import json
import shutil
from pydub import AudioSegment

if clear_data:
    if os.path.exists("/content/SOFA/data"):
        shutil.rmtree("/content/SOFA/data")

full_lab_path = "/content/SOFA/data/full_label"
weak_lab_path = "/content/SOFA/data/weak_label"
no_label = "/content/SOFA/data/no_label"
if not os.path.exists(full_lab_path):
    os.makedirs(full_lab_path)
if not os.path.exists(weak_lab_path):
    os.makedirs(weak_lab_path)
if not os.path.exists(no_label):
    os.makedirs(no_label)

if data_type == "full label (.lab)":
    !7z x "$data_zip_path" -o{full_lab_path}

# for funny auto dict generator lmao
out = "/content/SOFA/dictionary/custom_dict.txt"

phonemes = set()

def is_excluded(phoneme):
    return phoneme in ["pau", "AP", "SP", "sil"]

if data_type == "full label (.lab)":
    phoneme_folder_path = full_lab_path
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".lab"):
                fpath = os.path.join(root, file)
                with open(fpath, "r") as lab_file:
                    for line in lab_file:
                        line = line.strip()
                        if line:
                            phoneme = line.split()[2]
                            if not is_excluded(phoneme):
                                phonemes.add(phoneme)

with open(out, "w") as f:
    for phoneme in sorted(phonemes):
        f.write(phoneme + "	" + phoneme + "\n")

# for vowels and consonants.txt.... well adding luquid type for uta's script
dict_path = out
vowel_types = {"a", "i", "u", "e", "o", "N", "M", "NG"}
liquid_types = {"y", "w", "l", "r"} # r for english labels, it should be fine with jp too
vowel_data = []
consonant_data = []
liquid_data = []

with open(dict_path, "r") as f:
    for line in f:
        phoneme, _ = line.strip().split("\t")
        if phoneme[0] in vowel_types:
            vowel_data.append(phoneme)
        elif phoneme[0] in liquid_types:
            liquid_data.append(phoneme)
        else:
            consonant_data.append(phoneme)

vowel_data.sort()
liquid_data.sort()
consonant_data.sort()
directory = os.path.dirname(dict_path)

# make txt for language json file
vowel_txt_path = os.path.join(directory, "vowels.txt")
with open(vowel_txt_path, "w") as f:
    f.write(" ".join(vowel_data))
liquid_txt_path = os.path.join(directory, "liquids.txt")
with open(liquid_txt_path, "w") as f:
    f.write(" ".join(liquid_data))
consonant_txt_path = os.path.join(directory, "consonants.txt")
with open(consonant_txt_path, "w") as f:
    f.write(" ".join(consonant_data))


# here's a funny json append
with open(vowel_txt_path, "r") as f:
    vowel_data = f.read().split()
with open(liquid_txt_path, "r") as f:
    liquid_data = f.read().split()
with open(consonant_txt_path, "r") as f:
    consonant_data = f.read().split()
phones4json = {"vowels": vowel_data, "liquids": liquid_data}
with open("/content/nnsvs-db-converter/lang.sample.json", "w") as rawr:
    json.dump(phones4json, rawr, indent=4)

if data_type == "full label (.lab)":
    db_converter_script = "/content/nnsvs-db-converter/db_converter.py"
    for raw_folder_name in os.listdir(full_lab_path):
        raw_folder_path = os.path.join(full_lab_path, raw_folder_name)
        if os.path.isdir(raw_folder_path):
            if estimate_midi:
                !python {db_converter_script} -s {max_silence_phoneme_amount} -S 60 -l {segment_length} -m -c -L "/content/nnsvs-db-converter/lang.sample.json" --folder {raw_folder_path}
            else:
                #!python {db_converter_script} -s 2 --folder {raw_folder_path} 2> /dev/null
                !python {db_converter_script} -s {max_silence_phoneme_amount} -S 60 -l {segment_length} -L "/content/nnsvs-db-converter/lang.sample.json" --folder {raw_folder_path}

else:
    pass


if data_type == "full label (.lab)":
    for raw_folder_name in os.listdir(full_lab_path):
        raw_folder_path = os.path.join(full_lab_path, raw_folder_name)
        !rm -rf {raw_folder_path}/*.wav {raw_folder_path}/*.lab
        !mv {raw_folder_path}/diffsinger_db/* {raw_folder_path} 2> /dev/null
        !rm -rf {raw_folder_path}/diffsinger_db

else:
    pass

print("extraction complete!")

In [ ]:
#@title #Preprocess
#@markdown ___
%cd /content/SOFA

if not os.path.exists("data/binary/global_config.yaml"):
    os.makedirs("data/binary")
    with open("data/binary/global_config.yaml", "w") as file:
        pass

!source activate aligner; \
python binarize.py

In [ ]:
#@title #Train Your Model
#@markdown ___
#@markdown this is a rough draft notebook, you might have to edit the precision to 32-true in the config manually for now
%cd /content/SOFA

%reload_ext tensorboard
%tensorboard --logdir=ckpt/

!source activate aligner; \
python /content/SOFA/train.py -r